In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("worksheet3.ipynb")

# Worksheet 3: CRUD operations in MongoDB

Welcome! In this week, we will practice some basic CRUD (Create, Replace, Update, Delete) operations in MongoDB using Pymongo.

## Establish Pymongo connection

Recall that in worksheet1, we have created a cluster on MongoDB Atlas and connect to it via Pymongo and a credentials JSON file. Let's reestablish that connection for our exercises.

- Make sure you use the `adsc_3610` conda environment.
- You might need to copy & paste the `credentials_mongodb.json` file that you used in worksheet1 to the working directory of worksheet3.

In [ ]:
from pymongo import MongoClient # import mongo client to connect
import json # import json to load credentials
import urllib.parse

# load credentials from json file
with open('credentials_mongodb.json') as f:
    login = json.load(f)

# assign credentials to variables
username = login['username']
password = urllib.parse.quote(login['password'])
host = login['host']
url = "mongodb+srv://{}:{}@{}/?retryWrites=true&w=majority".format(username, password, host)

In [ ]:
# connect to the database
client = MongoClient(url)

In [ ]:
# drop database books and students if they exist
client.drop_database('books')
client.drop_database('students')

To test if your connection has been succesful, let's try to print out all the databases

In [ ]:
# list all databases
client.list_database_names()

## MongoDB VScode extension (optional)

If you are using VScode, there is a MongoDB extension which provides a handy view of the databases (instead of having to view it on MongoDB Atlas via a browser).

To install the extension, navigate to the extension bar in VS code and search for "mongodb"

![](img/vscode.png)

After installing the extension, you can now connect to your MongoDB Atlas cluster via the connection string.

![](img/connect.png)

To find your connection string, open a browser and navigate to MongoDB Atlas. 
- Under your cluster, click `connect`
- Select `MongoDB for VScode`
- Copy that URL and paste it in VScode

![](img/string.png)

Once connected, you should now be able to see an overview of your databases. Now you can manage your MongoDB and do programming all within VSCode. 

Now that we have establish the connection to our MongoDB cluster, let's get started with the exercises:

## Exercise 1: Import data into MongoDB

{rubric: accuracy = 5}

Let's first create a new database called `school` and store it in an object named `db`. We then create a new collection called `students`

In [ ]:
# Step 1: Connect to your MongoDB cluster
client = MongoClient(url)

# Step 2: Create a new database named 'school' and store it in a variable named db
db = client.school

# Step 3: Create a collection named 'students' and insert sample documents
students = db.students

Let's insert a sample student document to our `students` collection

In [ ]:
# Sample document to insert
sample_students = [{"name": "Alice", "age": 20, "major": "Computer Science"},
            {"name": "Bob", "age": 21, "major": "Mathematics"},
            {"name": "Charlie", "age": 22, "major": "Engineering"}]

# Insert the sample document
students.insert_many(sample_students)

You should see that the document has been inserted into our `students` collection, and it was assigned an `ObjectId`. 

> Note: Every document in MongoDB must have an `ObjectId`. If it doesn't exist, then MongoDB will create a new one automatically

Now we can print out the documents that we just inserted

In [ ]:
# Find the document that was just inserted
# The find_one() method returns the first occurrence in the selection.
students.find_one()

In [ ]:
# Find the document by a specific attribute
students.find_one({"name": "Bob"})

In [ ]:
# Find all documents in the collection
# The find() method returns all occurrences in the selection.
for student in students.find():
    print(student)

Now if you navigate to your databases in MongoDB Atlas, or using the VSCode MongoDB extension, you should see there's a new database called `school`. Within that database, there should be a collection named `students`, which contains one document about Alice

In [ ]:
# Try to find all documents where age is greater than 20
for student in students.find({"age": {"$gt": 20}}):
    print(student)

### YOUR TURN

### 1.1

- Create a new database called `bookstore`, stored it in a variable called `bookstore`
- Create a collection called `books`, stored it in a variable called `books`

In [ ]:
# Create a new database called `bookstore`, stored it in a variable called `bookstore`
# YOUR CODE HERE
bookstore = ...

# Create a collection called `books` 
# YOUR CODE HERE
books = ...

In [ ]:
sample_books = [
    {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald", "genre": "Fiction", "price": 10.99},
    {"title": "To Kill a Mockingbird", "author": "Harper Lee", "genre": "Fiction", "price": 8.99},
    {"title": "A Brief History of Time", "author": "Stephen Hawking", "genre": "Non-Fiction", "price": 15.99}
]

# YOUR CODE HERE

...

In [ ]:
# Print out a book with author as Stephen Hawking
# YOUR CODE HERE
...

In [ ]:
# Print out all books with price less than $10
# YOUR CODE HERE
...
    ...

Run the test below to see if you have done it correctly

In [ ]:
grader.check("ex1_1")

### 1.2 Import from JSON

Imagine we have a dataset called `students.json` and we would like to import it into the `students` collection in our MongoDB database.

In [ ]:
import json

# Step 1: Load JSON data
with open('students.json', 'r') as file:
    data = json.load(file)


In [ ]:
# view what the data looks like (first 5 records)
data[:5]

In [ ]:

# Step 2: Insert JSON data into a collection
students.insert_many(data)  # For a list of documents


In [ ]:
# Find all documents in the collection
# The find() method returns all occurrences in the selection.
for student in students.find():
    print(student)

Now it's your turn, import the `books.json` into the `books` collection in the `bookstore` database in your MongoDB.

In [ ]:
...

In [ ]:
grader.check("ex1_2")

## Exercise 2: Insert & update new field

Let's say we want to add a field called `gpa` to the `students` collection. Here's how

In [ ]:
# Step 1: Define the new field and its value
new_field = {"gpa": None}

# Step 2: Update all documents to include the new field
students.update_many({}, {"$set": new_field})

In [ ]:
# check if the new field was added
for student in students.find():
    print(student)

Let's try to update the `gpa` of Jack Daniels to 3.4

In [ ]:
# Step 2: Define the filter and the new value
filter = {"name": "Jack Daniels"}
new_value = {"$set": {"gpa": 3.4}}

# Step 3: Update the specific document
students.update_one(filter, new_value)

In [ ]:
# check Jack Daniels GPA
students.find_one({"name": "Jack Daniels"})



Now it's your turn. 
- Insert a field called `"publisher"` into the `books` collection with a default value of `"Unknown"`. 
- Update the `publisher` of the book with the author name is "Ray Bradbury", to "Penguin Books"

In [ ]:
...

In [ ]:
# print out the document by Ray Bradbury

In [ ]:
grader.check("ex2")

## Exercise 3: Replace

Let's print out our student Jack Daniels

In [ ]:
students.find_one({"name": "Jack Daniels"})

Let's say we want to replace the entire document by a new one. Let's called this `new_student`

In [ ]:
new_student = {
    "name": "John Doe", 
    "age": 25, 
    "major": "History", 
    "email": "john_doe@gmail.com",
    "gpa": 3.1
}

students.replace_one({"name": "Jack Daniels"}, new_student)

# check if Jack Daniels was replaced
students.find_one({"name": "Jack Daniels"})


In [ ]:
students.find_one({"name": "John Doe"})


Now it's your turn.

Replace the book by Stephen Hawking with the `new_book`

In [ ]:
new_book = {
    "title": "The Universe in a Nutshell",
    "author": "Stephen Hawking",
    "genre": "Non-Fiction",
    "price": 12.99,
    "publisher": "Bantam Books"
}

# YOUR CODE HERE
...


In [ ]:
# print out the document by Stephen Hawking
...

In [ ]:
grader.check("ex3")

## Exercise 4: Delete


Let's say I want to delete all documents in the `students` collection where their `age` is above 21

In [ ]:
# Step 1: Define the filter to find the documents to be deleted
filter = {"age": {"$gt": 21}}

# Step 2: Delete the documents
result = students.delete_many(filter)

# Step 3: Print the number of documents deleted
print(result.deleted_count, " documents deleted.")

# check if the documents were deleted
for student in students.find():
    print(student)

Now it's your turn.

Delete all the documents in the `books` collection where the `title` start with letter "T" 

> Hint: You can use regex expression in the filter. For example, if I want all students start with letter "M", I would use 
> 
> `filter = {"name": {"$regex": "^M"}}`

In [ ]:
# Delete all the documents in the `books` collection where the `title` start with letter "T"
# YOUR CODE HERE
...


In [ ]:
# Print the number of documents deleted
...

In [ ]:
# check if the documents were deleted
# find all books that start with letter "T"
...
    ...

In [ ]:
# print all the documents in the collection
...
    ...

In [ ]:
grader.check("ex4")

## Exercise 5: Visualize

Let's say that we are retriving students data and we want to convert the results to pandas DataFrame

In [ ]:
import pandas as pd
# Step 2: Query all documents in the collection
cursor = students.find()

# Step 3: Convert the cursor to a list and then to a DataFrame
students_data = list(cursor)
df = pd.DataFrame(students_data)

# Print the DataFrame to verify
df.head()

Now let's create a simple bar chart of students' age

In [ ]:
# create a histogram of students age
df['age'].plot(kind='hist', title='Age Distribution of Students')

Now it's your turn

- Convert the `books` collection into a pd.Dataframe called `df_books`
- Create a histogram of books' prices

In [ ]:
# Convert the `books` collection into a pd.Dataframe called `df_books`
# YOUR CODE HERE
...

In [ ]:
# visualize the distribution of book prices
...

In [ ]:
grader.check("ex5")

## Submission instructions

{rubric: mechanics = 5}

- Make sure the notebook can run from top to bottom without any error. Restart the kernel and run all cells.
- Commit and push your notebook to the github repo
- Double check your notebook is rendered properly on Github and you can see all the outputs clearly